# Python Gateway Tutorial

The UnetStack Python gateway API is available via [unet-contrib](https://github.com/org-arl/unet-contrib/tree/master/contrib/Unet-Python-API), or from [PyPi](https://pypi.org/project/unetpy/).

## Import unetpy

If you haven't installed unetpy, you need to do that first: `pip install unetpy`

In [1]:
from unetpy import *

## Import required messages

UnetStack3 provides the user with messages that can be used for implementing various functionalities. We are importing here some such message classes for demonstrating it's use.

In [2]:
TxFrameReq = MessageClass('org.arl.unet.phy.TxFrameReq')
TxBasebandSignalReq = MessageClass('org.arl.unet.bb.TxBasebandSignalReq')
RecordBasebandSignalReq = MessageClass('org.arl.unet.bb.RecordBasebandSignalReq')
TxFrameNtf = MessageClass('org.arl.unet.phy.TxFrameNtf')
RxFrameNtf = MessageClass('org.arl.unet.phy.RxFrameNtf')
RxBasebandSignalNtf = MessageClass('org.arl.unet.bb.RxBasebandSignalNtf')

## Open a connection to the modem or real-time simulator

For now, we'll assume that we have a modem running on localhost port 1100 (default):

In [3]:
modem = UnetGateway('localhost')

## Work with modem parameters

If we are connected to the modem, we can now access the agents and services that the modem provides. Let us try this with the physical layer first. What you'll see here depends on the modem you are using (we are using the portaudio modem on a laptop for this example).

In [4]:
phy = modem.agentForService(Services.PHYSICAL)

In [5]:
phy


[org.arl.unet.DatagramParam]
  MTU = 7

[org.arl.unet.bb.BasebandParam]
  basebandRate = 12000.0
  carrierFrequency = 12000.0
  maxPreambleID = 4
  maxSignalLength = 2147483647
  preambleDuration = 0.04
  signalPowerLevel = -10.0

[org.arl.unet.phy.PhysicalParam]
  busy = False
  maxPowerLevel = 0.0
  minPowerLevel = -138.0
  propagationSpeed = 1500.0
  refPowerLevel = 0.0
  rxEnable = True
  rxSensitivity = 0.0
  time = 6885397333
  timestampedTxDelay = 500

[org.arl.unet.scheduler.SchedulerParam]
  rtc = Dec 20, 2018 3:45:47 PM
  wakeOnAcoustic = False
  wakeOnEthernet = False
  wakeOnRS232 = False

[org.arl.yoda.ModemParam]
  bpfilter = True
  diag = none
  fan = False
  fanctl = 45.0
  fullduplex = False
  gain = 0.0
  inhibit = 120
  isc = True
  loopback = False
  model = portaudio
  mute = True
  noise = -59.7
  npulses = 1
  pbsblk = 65536
  pbscnt = 0
  poweramp = True
  preamp = True
  pulsedelay = 0
  serial = portaudio
  vendor = Subnero
  voltage = 0.0
  wakeupdelay = 400

We can query individual parameters or change them:

In [6]:
phy.signalPowerLevel

-10.0

In [7]:
phy.signalPowerLevel = -6

In [8]:
phy.signalPowerLevel

-6.0

We can work with the CONTROL (1) or DATA (2) channels too...

In [9]:
phy[1]


[org.arl.unet.DatagramParam]
  MTU = 7

[org.arl.unet.phy.PhysicalChannelParam]
  dataRate = 52.173912
  errorDetection = True
  fec = 3
  fecList = ['LDPC1', 'LDPC2', 'LDPC3', 'LDPC4', 'LDPC5', 'LDPC6']
  frameDuration = 1.84
  frameLength = 12
  maxFrameLength = 132
  powerLevel = -10.0

[org.arl.yoda.FhbfskParam]
  chiplen = 1
  fmin = 9520.0
  fstep = 160.0
  hops = 13
  tukey = True

[org.arl.yoda.ModemChannelParam]
  basebandExtra = 0
  basebandRx = False
  modulation = fhbfsk
  preamble = org.arl.yoda.Preamble(...)
  test = False
  threshold = 0.3
  valid = True


In [10]:
phy[1].frameLength = 12

In [11]:
phy[1].frameDuration

1.84

You can also work with higher layers:

In [12]:
link = modem.agentForService(Services.LINK)

In [13]:
link


[org.arl.unet.DatagramParam]
  MTU = 976

[org.arl.unet.link.ReliableLinkParam]
  dataChannel = 2
  mac = mac
  maxPropagationDelay = 2.5
  maxRetries = 2
  phy = phy
  reservationGuardTime = 0.5


## Send and receive messages

The messages supported on the Python gatweway are pretty much the same as the Java/Groovy messages. In Python, the named parameters for message initialization use equals (=) instead of colon (:), and you don't need the new keyword. It's easy to get used to:

In [14]:
phy << TxFrameReq(to=2, data=[1,2,3,4])

AGREE

And read the TxFrameNtf notification once the packet is sent out:

In [15]:
txntf = modem.receive(timeout=2000)

## Transmit and receive signals

For this part of the tutorial, we'll use numpy and arlpy. So if you don't have them installed, you'll need them: `pip install arlpy` (which will also install `numpy`).

In [16]:
import numpy as np
import arlpy.signal as asig
import arlpy.plot as plt

Generate a passband 100 ms 12 kHz pulse at a sampling rate of 96 kSa/s:

In [17]:
fs = 96000
x = asig.cw(12000, 0.1, fs)

and transmit it using the baseband service:

In [18]:
bb = modem.agentForService(Services.BASEBAND)
bb << TxBasebandSignalReq(signal=x, fc=0, fs=fs)

AGREE

In [19]:
txntf = modem.receive(timeout=2000)

By setting `fc` to `0`, we told the modem that this was a passband signal. The sampling rate supported by passband signals will depend on the modem. In our case, the portaudio interface is set to accept 96 kSa/s passband signals.

Now let's ask the modem to record a signal for us:

In [20]:
bb << RecordBasebandSignalReq(recLength=4800) 

AGREE

In [21]:
rec = modem.receive(RxBasebandSignalNtf, timeout=2000)

In [22]:
rec.fc

12000

In [23]:
rec.fs

12000

The notification has 4800 baseband (complex) samples as we had asked, and is sampled at a baseband rate of 12 kSa/s. The carrier frequency used by the modem is 12 kHz. We can convert our recorded signal to passband if we like:

In [28]:
y = asig.bb2pb(rec.signal, rec.fs, rec.fc, fs)

In [25]:
plt.plot(y, fs=fs)

In [26]:
plt.specgram(y, fs=fs)

## Clean up

Once we are done, we can clean up by closing the connection to the modem.

In [27]:
modem.close()

The gateway connection is closed!

The remote connection is closed!

